In [8]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os


In [17]:
load_dotenv()
groq_api_key_new = os.getenv("GROQ_API_KEY")

In [19]:
type(groq_api_key_new)

str

In [20]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key=groq_api_key_new,
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [21]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/principal-ai-ml-innovator-generative-ai-design/job/R-62107")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Principal AI/ML Innovator, Generative AI Design










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Retur

In [22]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [23]:
print(res.content)

```json
{
  "role": "Principal AI/ML Innovator, Generative AI Design",
  "experience": "8+ years in creative technology, applied AI/ML, or related roles",
  "skills": [
    "Generative AI image diffusion models and workflows",
    "Curating and preparing custom image datasets for model training and experimentation",
    "Strong collaborative mindset",
    "Excellent communication, stakeholder management, and leadership skills",
    "Deep curiosity about the evolving role of AI in creativity—and a commitment to building thoughtful, empowering tools for others",
    "Ability to influence build/buy decisions in AI & ML"
  ],
  "description": "We’re looking for a Principal AI/ML Innovator, Generative AI Design to join the Performance Engineering Lab, reporting to the Director of Advanced Product Intelligence. The Performance Engineering Lab, within the Advanced Digital Creation Studio, is focused on exploring new concepts for Nike products, such as footwear or apparel, and bringing underst

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Principal AI/ML Innovator, Generative AI Design',
 'experience': '8+ years in creative technology, applied AI/ML, or related roles',
 'skills': ['Generative AI image diffusion models and workflows',
  'Curating and preparing custom image datasets for model training and experimentation',
  'Strong collaborative mindset',
  'Excellent communication, stakeholder management, and leadership skills',
  'Deep curiosity about the evolving role of AI in creativity—and a commitment to building thoughtful, empowering tools for others',
  'Ability to influence build/buy decisions in AI & ML'],
 'description': 'We’re looking for a Principal AI/ML Innovator, Generative AI Design to join the Performance Engineering Lab, reporting to the Director of Advanced Product Intelligence. The Performance Engineering Lab, within the Advanced Digital Creation Studio, is focused on exploring new concepts for Nike products, such as footwear or apparel, and bringing understanding of product function using

In [25]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [15]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore1')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [24]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore_new')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    documents = []
    metadatas = []
    ids = []

    for _, row in df.iterrows():
        documents.append(row["Techstack"])
        metadatas.append({"links": row["Links"]})
        ids.append(str(uuid.uuid4()))

    collection.add(documents=documents, metadatas=metadatas, ids=ids)


In [25]:
links = collection.query(query_texts=["Experience in Python","Experience in React Native"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [26]:
job=json_res
job['skills']

['Generative AI image diffusion models and workflows',
 'Curating and preparing custom image datasets for model training and experimentation',
 'Strong collaborative mindset',
 'Excellent communication, stakeholder management, and leadership skills',
 'Deep curiosity about the evolving role of AI in creativity—and a commitment to building thoughtful, empowering tools for others',
 'Ability to influence build/buy decisions in AI & ML']

In [27]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [28]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlocking Generative AI Capabilities with AtliQ

Dear Hiring Manager,

I came across the job description for a Principal AI/ML Innovator, Generative AI Design at Nike, and I was impressed by the company's commitment to exploring new concepts and bringing innovative products to market. As a business development executive at AtliQ, I believe our team can help Nike unlock new creative potential with our expertise in AI and ML.

At AtliQ, we have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team has extensive experience in developing and implementing Generative AI image diffusion models and workflows, curating and preparing custom image datasets for model training and experimentation, and collaborating with cross-functional teams to deliver innovative solutions.

I'd like to highlight some of our relevant portfolio work that demonstrates our capabilities in AI and ML:

*

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Upal, a 4th year undergraduate student of the Department of Industrial and Systems Engineering at IIT Kharagpur . I have developed a strong passion for Data Analysis, Machine Learning, Deep Learning, and Generative AI. My academic journey has equipped me with the skills to leverage data-driven solutions for process optimization and informed decision-making. I am eager to connect with industry professionals and explore opportunities for collaboration and innovation in these critical fields.
        Your job is to write a cold email to the company regarding the job mentioned for applying for the job.
        Also add the most relevant ones from the following links to showcase your portfolio: {link_list}
        Remember you are Upal, a 4th year undergraduate student of the Department of Industrial and Systems Engineering at IIT Kharagpur. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Principal AI/ML Innovator, Generative AI Design Role

Dear Hiring Manager,

I am excited to apply for the Principal AI/ML Innovator, Generative AI Design position at Nike, as advertised. With a strong passion for Data Analysis, Machine Learning, Deep Learning, and Generative AI, I am confident that my skills and academic background make me an ideal candidate for this role.

As a 4th year undergraduate student of the Department of Industrial and Systems Engineering at IIT Kharagpur, I have developed a solid foundation in leveraging data-driven solutions for process optimization and informed decision-making. My academic journey has equipped me with the skills to work on complex projects, and I am eager to bring my expertise to a dynamic team like yours.

I would like to showcase my relevant portfolio, which demonstrates my proficiency in Machine Learning and Python. You can view my projects at https://example.com/ml-python-portfolio. I believe that my experience 

: 